In [3]:
import torch
import random

generic_linear_transform = torch.randn(8, 1)
print(f"generic: {generic_linear_transform}", sep="\n")

local_linear_transform = torch.tensor([
    [0, 0, 0] + [random.random(), random.random(), random.random()] + [0, 0]]).T
print("local:", local_linear_transform, sep="\n")

generic: tensor([[ 0.7265],
        [ 0.1867],
        [ 0.7819],
        [ 0.3247],
        [ 0.1036],
        [-0.6590],
        [-0.2059],
        [-0.5688]])
local:
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.3458],
        [0.2975],
        [0.9182],
        [0.0000],
        [0.0000]])


In [4]:
generic_linear_transform = torch.arange(8)[:, None]
print("generic:", generic_linear_transform, sep="\n")

equivariant_linear_transform = torch.stack([torch.roll(generic_linear_transform[:, 0], ii) for ii in range(8)], dim=1)
print("translation invariant:", equivariant_linear_transform, sep="\n")

generic:
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])
translation invariant:
tensor([[0, 7, 6, 5, 4, 3, 2, 1],
        [1, 0, 7, 6, 5, 4, 3, 2],
        [2, 1, 0, 7, 6, 5, 4, 3],
        [3, 2, 1, 0, 7, 6, 5, 4],
        [4, 3, 2, 1, 0, 7, 6, 5],
        [5, 4, 3, 2, 1, 0, 7, 6],
        [6, 5, 4, 3, 2, 1, 0, 7],
        [7, 6, 5, 4, 3, 2, 1, 0]])


In [12]:
# the equivalent of torch.nn.Linear, but for a 1-dimensional convolution
conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

conv_layer.weight  # aka kernel

Parameter containing:
tensor([[[ 0.1716, -0.0264,  0.4943]]], requires_grad=True)

In [20]:
conv_kernel_as_vector = torch.hstack([conv_layer.weight[0][0], torch.zeros(5)])
conv_layer_as_matrix = torch.stack([torch.roll(conv_kernel_as_vector, ii) for ii in range(8)], dim=0)
print("convolution matrix:", conv_layer_as_matrix, sep="\n")
print(f"conv_kernel_as_vector: {conv_kernel_as_vector}",sep='\n')

convolution matrix:
tensor([[ 0.1716, -0.0264,  0.4943,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.1716, -0.0264,  0.4943,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.1716, -0.0264,  0.4943,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.1716, -0.0264,  0.4943,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.1716, -0.0264,  0.4943,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1716, -0.0264,  0.4943],
        [ 0.4943,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1716, -0.0264],
        [-0.0264,  0.4943,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1716]],
       grad_fn=<StackBackward0>)
conv_kernel_as_vector: tensor([ 0.1716, -0.0264,  0.4943,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       grad_fn=<CatBackward0>)


In [24]:
from IPython.display import IFrame
import IPython.display
layers = ["conv2d0", "conv2d1", "conv2d2", "mixed3a", "mixed3b"]
layer = layers[1]
idx = 52

weight_explorer = display.IFrame(
    src=f"https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/{layer}_{idx}.html", width=1024, height=720)
weight_explorer.iframe = 'style="background: #FFF";\n><'.join(weight_explorer.iframe.split("><"))  # inject background color
weight_explorer

AttributeError: 'function' object has no attribute 'IFrame'

In [25]:
import text_recognizer.models


text_recognizer.models.CNN??

Init signature:
text_recognizer.models.CNN(
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Source:        
class CNN(nn.Module):
    """Simple CNN for recognizing characters in a square image."""

    def __init__(self, data_config: Dict[str, Any], args: argparse.Namespace = None) -> None:
        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_channels, input_height, input_width = self.data_config["input_dims"]
        assert (
            input_height == input_width
        ), f"input height and width should be equal, but was {input_height}, {input_width}"
        self.input_height, self.input_width = input_height, input_width

        num_classes = len(self.data_config["mapping"])

        conv_dim = self.args.get("conv_dim", CONV_DIM)
        fc_dim = self.args.get("fc_dim", FC_DIM)
        fc_dropout = self.args.get("fc_dropout", FC_DROPOUT)

        self.conv1 = Co

In [27]:
import text_recognizer.data


emnist = text_recognizer.data.EMNIST()  # configure
print(emnist.__doc__)

EMNIST dataset of handwritten characters and digits.

    "The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19
    and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset."
    From https://www.nist.gov/itl/iad/image-group/emnist-dataset

    The data split we will use is
    EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
    


In [28]:
emnist.prepare_data()  # download, save to disk
emnist.setup()  # create torch.utils.data.Datasets, do train/val split

709MB [05:22, 2.30MB/s]                               


Computing SHA-256...
Unzipping EMNIST...
Loading training data from .mat file
Balancing classes to reduce amount of data
Saving to HDF5 in a compressed format...
Saving essential dataset parameters to text_recognizer/data...
Cleaning up...


In [29]:
!echo {emnist.data_dirname()}
!ls {emnist.data_dirname()}
!ls {emnist.data_dirname() / "raw" / "emnist"}

/home/stanley/Documents/shidder/ml/FullStackDL/fsdl-2022-labs/data
downloaded  processed  raw
metadata.toml  readme.md


In [30]:
emnist

EMNIST Dataset
Num classes: 83
Mapping: ['<B>', '<S>', '<E>', '<P>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '?']
Dims: (1, 28, 28)
Train/val/test sizes: 260212, 65054, 53988
Batch x stats: (torch.Size([128, 1, 28, 28]), torch.float32, tensor(0.), tensor(0.1706), tensor(0.3307), tensor(1.))
Batch y stats: (torch.Size([128]), torch.int64, tensor(4), tensor(64))

In [31]:
xs, ys = next(iter(emnist.train_dataloader()))

In [35]:
xs.shape

torch.Size([128, 1, 28, 28])

In [74]:
import wandb

idx = random.randint(0, len(xs) - 1)

print(emnist.mapping[ys[idx]])
wandb.Image(xs[idx]).image

d


In [75]:
data_config = emnist.config()

cnn = text_recognizer.models.CNN(data_config)
cnn  # reveals the nn.Modules attached to our nn.Module

CNN(
  (conv1): ConvBlock(
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (dropout): Dropout(p=0.25, inplace=False)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=83, bias=True)
)

In [102]:
idx = random.randint(0, len(xs) - 1)
outs = cnn(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: y


In [103]:
cnn.forward??


Signature: cnn.forward(x: torch.Tensor) -> torch.Tensor
Source:   
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Applies the CNN to x.

        Parameters
        ----------
        x
            (B, Ch, H, W) tensor, where H and W must equal input height and width from data_config.

        Returns
        -------
        torch.Tensor
            (B, Cl) tensor
        """
        _B, _Ch, H, W = x.shape
        assert H == self.input_height and W == self.input_width, f"bad inputs to CNN with shape {x.shape}"
        x = self.conv1(x)  # _B, CONV_DIM, H, W
        x = self.conv2(x)  # _B, CONV_DIM, H, W
        x = self.max_pool(x)  # _B, CONV_DIM, H // 2, W // 2
        x = self.dropout(x)
        x = torch.flatten(x, 1)  # _B, CONV_DIM * H // 2 * W // 2
        x = self.fc1(x)  # _B, FC_DIM
        x = F.relu(x)
        x = self.fc2(x)  # _B, Cl
        return x
File:      ~/Documents/shidder/ml/FullStackDL/fsdl-2022-labs/lab02/text_recognizer/models/cnn.py
Type

In [104]:
torch.flatten(torch.Tensor([[1, 2], [3, 4]]))

tensor([1., 2., 3., 4.])

In [105]:
[p.numel() for p in cnn.parameters()]  # conv weight + bias, conv weight + bias, fc weight + bias, fc weight + bias

[576, 64, 36864, 64, 1605632, 128, 10624, 83]

In [106]:
biggest_layer = [p for p in cnn.parameters() if p.numel() == max(p.numel() for p in cnn.parameters())][0]
biggest_layer.shape, cnn.fc_input_dim

(torch.Size([128, 12544]), 12544)

In [110]:
# for the Linear layers, number of multiplications per input == nparams
cnn.fc1.weight.numel()

1605632

In [108]:
# for the Conv2D layers, it's more complicated

def approx_conv_multiplications(kernel_shape, input_size=(32, 28, 28)):  # this is a rough and dirty approximation
    num_kernels, input_channels, kernel_height, kernel_width = kernel_shape
    input_height, input_width = input_size[1], input_size[2]

    multiplications_per_kernel_application = input_channels * kernel_height * kernel_width
    num_applications = ((input_height - kernel_height + 1) * (input_width - kernel_width + 1))
    mutliplications_per_kernel = num_applications * multiplications_per_kernel_application

    return mutliplications_per_kernel * num_kernels

In [111]:
approx_conv_multiplications(cnn.conv2.conv.weight.shape)

24920064

In [112]:
# ratio of multiplications in the convolution to multiplications in the fully-connected layer is large!
approx_conv_multiplications(cnn.conv2.conv.weight.shape) // cnn.fc1.weight.numel()

15

In [113]:
%run training/run_experiment.py --help

AttributeError: type object 'Trainer' has no attribute 'add_argparse_args'

In [114]:
gpus = int(torch.cuda.is_available())  # use GPUs if they're available

%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus}

AttributeError: type object 'Trainer' has no attribute 'add_argparse_args'

In [115]:
from text_recognizer.data.sentence_generator import SentenceGenerator

sentence_generator = SentenceGenerator()

SentenceGenerator.__doc__

print(*[sentence_generator.generate(max_length=16) for _ in range(4)], sep="\n")

phase
Within
either
the


In [116]:
emnist_lines = text_recognizer.data.EMNISTLines()  # configure
emnist_lines.__doc__

'EMNIST Lines dataset: synthetic handwriting lines dataset made from EMNIST characters.'

In [117]:
emnist_lines.prepare_data()  # download, save to disk
emnist_lines.setup()  # create torch.utils.data.Datasets, do train/val split
emnist_lines

EMNISTLinesDataset loading data from HDF5...


EMNIST Lines Dataset
Min overlap: 0
Max overlap: 0.33
Num classes: 83
Dims: (1, 28, 896)
Output dims: (32, 1)
Train/val/test sizes: 10000, 2000, 2000
Batch x stats: (torch.Size([128, 1, 28, 896]), torch.float32, 0.0, 0.07314881682395935, 0.23142141103744507, 1.0)
Batch y stats: (torch.Size([128, 32]), torch.int64, 3, 66)

In [118]:
line_xs, line_ys = next(iter(emnist_lines.val_dataloader()))
line_xs.shape, line_ys.shape

(torch.Size([128, 1, 28, 896]), torch.Size([128, 32]))

In [122]:
def read_line_labels(labels):
    return [emnist_lines.mapping[label] for label in labels]

idx = random.randint(0, len(line_xs) - 1)

print("-".join(read_line_labels(line_ys[idx])))
wandb.Image(line_xs[idx]).image

m-e-t-e-r-s- -a-n-d- -l-e-a-s-i-n-g- -t-h-e-s-e- -f-o-r-<P>-<P>-<P>-<P>


In [123]:
line_cnn = text_recognizer.models.LineCNNSimple(emnist_lines.config())
line_cnn

LineCNNSimple(
  (cnn): CNN(
    (conv1): ConvBlock(
      (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (conv2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=12544, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=83, bias=True)
  )
)

In [124]:
line_cnn.forward??

Signature: line_cnn.forward(x: torch.Tensor) -> torch.Tensor
Source:   
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Apply the LineCNN to an input image and return logits.

        Parameters
        ----------
        x
            (B, C, H, W) input image with H equal to IMAGE_SIZE

        Returns
        -------
        torch.Tensor
            (B, C, S) logits, where S is the length of the sequence and C is the number of classes
            S can be computed from W and CHAR_WIDTH
            C is self.num_classes
        """
        B, _C, H, W = x.shape
        assert H == IMAGE_SIZE  # Make sure we can use our CNN class

        # Compute number of windows
        S = math.floor((W - self.WW) / self.WS + 1)

        # NOTE: type_as properly sets device
        activations = torch.zeros((B, self.num_classes, S)).type_as(x)
        for s in range(S):
            start_w = self.WS * s
            end_w = start_w + self.WW
            window = x[:, :, :, start_

In [126]:
idx = random.randint(0, len(line_xs) - 1)

outs, = line_cnn(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

N-L-L-T-1-N-N-T-N-N-N-L-N-N-N-N-L-L-N-N-G-N-L-N-N-T-N-N-N-N-N-N
